In [ ]:
import requests as r
from Bio import SeqIO
from io import StringIO

# get a list of unirprot IDs to try to find the sequence for
uniprot_id_list = []

# dictionary of Seq records
seq_records_dict = {}
# list of IDs not found
not_uniprot_list = []

# I use the counter to measure the progress since it is fairly slow
count = 0

for seq_id in df_missing["Protein ID"].tolist():
    
    # progress counter will print every 100 sequences
    count+=1
    if count % 100 == 0:
        print(count)
        
    # call the unisave and return text to parse
    baseUrl="http://www.uniprot.org/unisave/"
    currentUrl=baseUrl+seq_id+".fasta"
    response = r.post(currentUrl)

    # if uniprot ID not found then skip
    if response.status_code == 400:
        not_uniprot_list.append(seq_id)
        continue

    # remove all "\nRAW" for returned text
    cData=''.join(response.text).replace("\nRAW", "")
    
    # use biopython to parse the output
    Seq=StringIO(cData)
    # The first item of the list is the latest versison
    record=list(SeqIO.parse(Seq,'fasta'))[0]
    # I prefer the shorter id version, not original fasta id
    record_id = record.id.split("|")[1]
    record.id = record_id
    # I prefer to remove all information except id and seq
    record.name = ""
    record.description = ""
    
    # dictionary stores complete Seq record, not just sequence
    seq_records_dict[record_id] = record